<a href="https://colab.research.google.com/github/michelleaeh/CE888_Laboratory/blob/master/Laboratory5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Lab 5**

 Load the data from "jester-data-1.csv"

In [0]:
import numpy as np
import pandas as pd

df=pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv")

Using the helper function provided in the helper_function notebook (or create your own), label 10% of the dataset cells as 99. This is your validation set. Keep the the actual values of the cells so you can use them later.

In [0]:
arr=df.values[:,1:]
rated=np.where(arr!=99)
def replace(orig, percentage=0.1):
  new_data=orig.copy()
  rated=np.where(arr!=99)
  n_rated=len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])
new_arr, idx = replace(arr, 0.1)

Use latent factor modeling to infer the hidden ratings of the users (they are labeled as "99" in the dataset) on the training set

In [0]:
n_latent_factors=2
user_ratings=arr
latent_user_preferences=np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features=np.random.random((user_ratings.shape[1],n_latent_factors))

In [0]:
def predict_rating(user_id,item_id):
  """ Predict a rating given a user_id and an item_id.
  """
  user_preference = latent_user_preferences[user_id]
  item_preference = latent_item_features[item_id]
  return user_preference.dot(item_preference)

In [0]:
def train(user_id, item_id, rating, alpha=0.0001):
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err

In [0]:
def sgd(iterations=100):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating !=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        if (iteration % 10) == 0:
            print(mse)

Calculate the performance (e.g., MSE) of the algorithm on the validation dataset

In [7]:
sgd()

26.158501871221365
23.04100054918432
20.936329526338575
18.167570299632356
17.33115350774443
17.1886108916556
17.148647830196673
17.136111840562627
17.132236111284705
17.131268728933605


In [8]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(4.08, 2.9825524195219564)","(-0.29, 3.147925798578328)","(6.36, 3.7847563156423925)","(4.37, 5.264543239269295)","(-2.38, 2.2072508777035926)","(-9.66, 2.5176933829899784)","(-0.73, 2.9654991853617063)","(-5.34, 2.3256308654322044)","(8.88, 5.071567752562926)","(9.22, 2.992408878611676)","(6.75, 2.3875590199669583)","(8.64, 2.5594577935805867)","(4.42, 3.1407485619027744)","(7.43, 2.4793266083814465)","(4.56, 3.015379125739115)","(-0.97, 4.308795902425137)","(4.66, 2.448651479080305)","(-0.68, 2.9442284780256487)","(3.3, 2.718734928328418)","(-1.21, 3.5755488163711)","(0.87, 1.6207467536041225)","(8.64, 3.4853837361706064)","(8.35, 4.01175144068725)","(9.17, 5.512420917868451)","(0.05, 3.5335839022517934)","(7.57, 2.424058256667754)","(4.71, 0.23970409281373295)","(0.87, 2.3929837647790237)","(-0.39, 0.1242094204414823)","(6.99, 4.866198122734438)","(6.5, 1.250924444371441)","(-0.92, 0.3292072305709619)","(7.14, 5.210436978152182)","(9.03, 2.766454308550508)","(-1.8, 0.5271176398220552)","(0.73, 0.05356573735521089)","(7.09, 4.851441693859095)","(3.4, 3.0635936101703867)","(-0.87, 2.7850491401483657)","(7.91, 3.2151457275381086)",...,"(-6.7, 1.3069699645453052)","(-3.35, 0.8883291186605242)","(-9.03, 3.788047944114016)","(4.47, 4.717926443302505)","(4.08, 1.1537208976073907)","(-3.83, 1.0755383899284359)","(8.74, 4.950265268317908)","(1.12, 0.9509487087406375)","(0.78, 1.015178957988004)","(7.52, 3.5327923138915103)","(-5.0, 3.9772833037561917)","(2.77, 1.3018659907066035)","(8.3, 3.1056653226586355)","(7.77, 4.958885985673878)","(7.33, 3.791313725089856)","(6.21, 1.4481167488202278)","(7.72, 3.327921460575526)","(8.98, 2.720766321833954)","(8.64, 3.7911756267515027)","(8.2, 2.880933553926651)","(3.93, 2.2323861903934588)","(4.85, 3.525048040029215)","(4.85, 1.8550271800401699)","(6.07, 3.583482900861823)","(8.98, 3.4077034364072922)","(4.51, 3.8842867239257113)","(-0.05, 2.064033072841951)","(3.69, 2.155331187191222)","(4.56, 0.41543939480698866)","(0.58, 3.7920250308663754)","(2.82, 2.21710969281365)","(-4.95, 2.9269017570819895)","(-0.29, 1.6048589428097733)","(7.86, 3.481049812125127)","(-0.19, 3.071764076534814)","(-2.14, 2.7774431517518625)","(3.06, 2.423893481275884)","(0.34, 2.511911360753978)","(-4.32, 3.9598873108135337)","(1.07, 2.757415699690561)"
1,"(99.0, 6.857315051246974)","(99.0, 7.150771677836657)","(99.0, 7.437822147913523)","(99.0, 7.152137968810922)","(9.03, 5.49616133710191)","(9.27, 8.649181596663805)","(9.03, 4.915874414880194)","(9.27, 2.975089796102906)","(99.0, 7.540930060418719)","(99.0, 7.24584456075304)","(7.33, 8.362850523172975)","(7.57, 8.929132205949472)","(9.37, 2.7891573333575916)","(6.17, 8.861016594634698)","(-6.36, 2.076630831633803)","(-6.89, 1.7814618859263665)","(-7.86, 3.3280572402649446)","(9.03, 4.101715546706653)","(9.03, 5.7850549684598445)","(9.03, 5.557893821428852)","(7.28, 8.505684149294602)","(99.0, 8.511985729533407)","(8.25, 7.664816861107347)","(99.0, 6.418911639442241)","(99.0, 8.122219949815225)","(7.48, 8.59757036178594)","(7.28, 7.547945952698403)","(7.28, 8.22395892436454)","(8.93, 7.124117475464394)","(99.0, 8.284486764189959)","(6.17, 8.035958287493976)","(7.28, 7.611369220819026)","(99.0, 7.107163908716361)","(99.0, 8.125063647575805)","(8.98, 7.835633205750747)","(7.33, 7.125686006866061)","(99.0, 5.96831272529169)","(6.17, 8.084933680723067)","(9.08, 8.5660828843517)","(7.33, 8.422794898626949)",...,"(6.46, 8.467845774637189)","(7.28, 8.153930722485526)","(99.0, 7.88099738729279)","(99.0, 6.840356496013831)","(7.04, 8.010442548799796)","(7.28, 7.96343455907971)","(99.0, 7.050399368879119)","(7.28, 7.764774789988935)","(8.25, 8.124671202549345)","(99.0, 8.063317408911598)","(99.0, 6.152396877073469)","(99.0, 8.34555096133686)","(

Repeat the two points above changing hyper-parameters (i.e., learning rate, number of iterations of SVD, number of latent factors, etc.) as needed to get good results (you can create multiple validation sets if you want, and run a bootstrap!)

In [9]:
n_latent_factors=2
user_ratings=arr
latent_user_preferences=np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features=np.random.random((user_ratings.shape[1],n_latent_factors))
def predict_rating(user_id,item_id):
  """ Predict a rating given a user_id and an item_id.
  """
  user_preference = latent_user_preferences[user_id]
  item_preference = latent_item_features[item_id]
  return user_preference.dot(item_preference)
def train(user_id, item_id, rating, alpha=0.000001):
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
def sgd(iterations=100):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating !=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        if (iteration % 10) == 0:
            print(mse)
sgd()
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

27.687622854196484
27.16980346687219
26.768060175896075
26.45440330680993
26.20782861839372
26.01248129363494
25.856342837453184
25.730274736078158
25.62731224636251
25.542137561900677


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(4.08, 1.0691658295474025)","(-0.29, 0.31361977036001965)","(6.36, 0.7390096355918101)","(4.37, -0.8887407254874888)","(-2.38, 0.4975863005840139)","(-9.66, 1.3850614659096652)","(-0.73, -0.2531244774284475)","(-5.34, -0.48818063288989877)","(8.88, -0.20852267739530375)","(9.22, 1.3376404367380634)","(6.75, 1.811004104069274)","(8.64, 1.5932616555733754)","(4.42, -1.3646454851398018)","(7.43, 1.4293462120564782)","(4.56, -1.3144940430559264)","(-0.97, -2.8185700169244203)","(4.66, -0.8861536055515773)","(-0.68, -0.5385380769505099)","(3.3, 0.25561430470704344)","(-1.21, -0.5696421999840795)","(0.87, 2.093247918981605)","(8.64, 1.1685489729417828)","(8.35, 0.48389946565727116)","(9.17, -1.1277653827755236)","(0.05, 0.5768202423781887)","(7.57, 1.4081823217053095)","(4.71, 3.149635456695368)","(0.87, 1.5632016681327163)","(-0.39, 3.0120297556448015)","(6.99, -0.1125579538730835)","(6.5, 2.1694782342625785)","(-0.92, 3.0132851996409813)","(7.14, -0.6983785525830699)","(9.03, 0.9972648449128003)","(-1.8, 2.8887508865220743)","(0.73, 3.2711747057864033)","(7.09, -0.6718941745385539)","(3.4, 1.3234331358159424)","(-0.87, 1.201081507899554)","(7.91, 1.0268609159716195)",...,"(-6.7, 2.389281327811228)","(-3.35, 3.021108279854274)","(-9.03, 0.5816648142391753)","(4.47, -0.14945443336205644)","(4.08, 2.270592801716772)","(-3.83, 2.4500713209750122)","(8.74, -0.4861141162089104)","(1.12, 2.60893836559085)","(0.78, 2.645107331838476)","(7.52, 0.47700486847478557)","(-5.0, 0.4231575547585995)","(2.77, 1.5153841032019777)","(8.3, 1.3147151361908573)","(7.77, -0.3462813587035894)","(7.33, 0.569787809281253)","(6.21, 1.3730611496951595)","(7.72, 0.9400176803397725)","(8.98, 1.3446478778402229)","(8.64, 0.5963346435272979)","(8.2, 1.2128862048637912)","(3.93, 1.112065021503999)","(4.85, 0.8553424132133569)","(4.85, 1.6276826172477863)","(6.07, 0.7757488234936892)","(8.98, 1.0599801445286654)","(4.51, 0.34777313036582563)","(-0.05, 1.496074759337325)","(3.69, 1.7738349285058879)","(4.56, 2.21243033164084)","(0.58, 0.8950975981163743)","(2.82, 1.3032529268375221)","(-4.95, 1.1943218489248737)","(-0.29, 1.7578829435090615)","(7.86, 1.0625319996766516)","(-0.19, 0.9577539236853514)","(-2.14, 1.5598276714399235)","(3.06, 1.399567490955521)","(0.34, 0.9538464133300673)","(-4.32, 0.30731221715250673)","(1.07, 1.2524125171936424)"
1,"(99.0, 0.9650303828994039)","(99.0, 0.254067125172193)","(99.0, 0.6889504283228707)","(99.0, -0.900205968084238)","(9.03, 0.5277324852895764)","(9.27, 1.3259364329427459)","(9.03, -0.25677722432820205)","(9.27, -0.47562748185312675)","(99.0, -0.10553312025041832)","(99.0, 1.3935280883474324)","(7.33, 1.8355575046813397)","(7.57, 1.546482000560931)","(9.37, -1.34960601455839)","(6.17, 1.3693848462357472)","(-6.36, -1.2194940709231448)","(-6.89, -2.6929153076965546)","(-7.86, -0.8345176842659089)","(9.03, -0.5350279781203066)","(9.03, 0.2660628848275613)","(9.03, -0.5469174128952625)","(7.28, 1.9779454620615442)","(99.0, 1.1117520737226303)","(8.25, 0.46511506385637086)","(99.0, -1.0839763009972565)","(99.0, 0.5255317909943411)","(7.48, 1.2396720437563438)","(7.28, 3.020311391724363)","(7.28, 1.621054250956358)","(8.93, 2.8596809994075727)","(99.0, -0.01504705316003916)","(6.17, 2.1637240480920843)","(7.28, 2.86696463367348)","(99.0, -0.5689172054902946)","(99.0, 1.0647181767414153)","(8.98, 2.733220964473345)","(7.33, 3.1823191401340094)","(99.0, -0.7276779301196269)","(6.17, 1.2554440304469074)","(9.08, 1.0330353190080686)","(7.33, 1.011639364133861)",...,"(6.46, 2.353435880445855)","(7.28, 2.908047650887383)","(99.0, 0.5334830600288439)","(99.0, -0.07413005905895124)","(7.04, 2.2768222640153217)","(7.28, 2.331530232736008)","(99.0, -0.39405717069736806)","(7.28, 2.524322879454382)","(8.2

*OBSERVATIONS*
*   Increasing alpha to 0.01 returns nan MSE 
*   Reducing alpha to 0.000001 returns larger MSE
*   Increasing the number of latent factors to 7 converges the MSE at the same 5th value





Once you're happy, make predictions for the test dataset

(If you have time) Use pandas to find the best- and the worst-rated jokes